## WaveFunctionCollapse generation demo

#### NOTE: please move this notebook to the root folder (so it can import `wfd.scmap`)

In [ ]:
from wfd.scmap import randomize_subtiles, get_cv5_data, process_input_maps, get_shrink_mapping
from wfd.scmap import get_default_output_map_data, replace_tile_data

# input maps (must be same tileset)
# these are blizzard maps, you can find them in starcraft remastered install folder
maps = [
    "train_data/maps/platform/(2)Bottleneck.scm",
    "train_data/maps/platform/(2)Boxer.scm",
    "train_data/maps/platform/(2)Space Madness.scm",
    "train_data/maps/platform/(4)Blood Bath.scm",
    "train_data/maps/platform/(4)Nightmare Station.scm",
    "train_data/maps/platform/(4)Orbital Relay.scm",
    "train_data/maps/platform/(4)Tarsonis Orbital.scm",
    "train_data/maps/platform/(6)Ground Zero.scm",
    "train_data/maps/platform/(6)New Gettysburg.scm",
    "train_data/maps/platform/(8)Bridge to Bridge '98.scm",
    "train_data/maps/platform/(8)Orbital Death.scm",
    "train_data/maps/platform/(8)Station Unrest.scm",
]

# merging subtiles performs better on natural maps
# may hit problems on blended terrains if they use specific subtiles.
merge_subtiles = True

tileset, sc_to_gen, gen_to_sc, \
tile_count, conn_probs, freqs = process_input_maps(maps, merge_subtiles = merge_subtiles)

In [ ]:
# map tile numbers to a smaller scale to speed up generation
gen_to_shrinked, shrinked_to_gen, shrinked_count = get_shrink_mapping(freqs, is_freq_table = True)
conn_probs_shrinked = conn_probs[shrinked_to_gen, :, :][:, :, shrinked_to_gen]
freqs_shrinked = freqs[shrinked_to_gen]

In [ ]:
import numpy as np
cv5_data = get_cv5_data(tileset)

# customize probability using cv5 data
tile_weights = np.zeros((len(cv5_data),))
walkable = [d["all_walkable"] for d in cv5_data]
buildable = [d["buildable"] for d in cv5_data]
is_null = [d["null"] for d in cv5_data]

# define weights for different type of tiles
tile_weights[:] = 32
tile_weights[walkable] = 32
tile_weights[buildable] = 256
tile_weights[is_null] = 1

gen_probs = tile_weights[gen_to_sc]
tile_probs = gen_probs[shrinked_to_gen]
tile_probs /= np.sum(tile_probs)

In [ ]:
grass_tile = gen_to_shrinked[sc_to_gen[[8 * 16, 9 * 16]]]
land_tile = gen_to_shrinked[sc_to_gen[[2 * 16, 3 * 16]]]
highland_tile = gen_to_shrinked[sc_to_gen[[8 * 16, 9 * 16]]]

# the following commented-out code is a sample to precondition part of the map to be buildable

# buildable_tiles = np.unique(gen_to_shrinked[sc_to_gen[buildable]])[1:] # need to remove null tile (0)

# def conv(*val):
#     # coordinates to grids
#     return tuple(v//32 for v in val)

# def init_gen(generator):
#     # set all main mineral areas to buildable tiles only
#     generator.set_possible_states(buildable_tiles, region = conv(96, 32, 672, 480))
#     generator.set_possible_states(buildable_tiles, region = conv(2048, 64, 2656, 480))
#     generator.set_possible_states(buildable_tiles, region = conv(3392, 96, 4000, 512))
#     generator.set_possible_states(buildable_tiles, region = conv(3360, 2400, 3968, 2816))
#     generator.set_possible_states(buildable_tiles, region = conv(3392, 3552, 4000, 3968))
#     generator.set_possible_states(buildable_tiles, region = conv(1760, 3584, 2368, 4000))
#     generator.set_possible_states(buildable_tiles, region = conv(128, 3488, 704, 3936))
#     generator.set_possible_states(buildable_tiles, region = conv(64, 1344, 640, 1792))
#     generator.set_possible_states(buildable_tiles, region = conv(1728, 1952, 2304, 2400))
#     generator.process_initial_states(depth = 2)

In [ ]:
from wfd.wfc import WFCGenerator

size = (64, 64)                                       # size of generated map, can differ from input
generator = WFCGenerator(size,
                         conn_probs_shrinked,
                         tile_probs = tile_probs,     # tile_probs = freqs_shrinked to match input frequencies
                         use_uniform_probs = False,   # bypasses all probs calculation, might be faster
                         use_border_probs = False)    # it is a bad idea

# init_gen(generator)
# generator.set_possible_states(land_tile, region = (0, 0, 192, 4))
# generator.process_initial_states(depth = 2)

# try messing with these parameters if the algorithm fails to generate the map or takes forever

# lambda retries: (max_iterations, max_wait_after_stalls)
param_scheduler = lambda retries: (500 + 500 * min(10, retries), 200 + 200 * min(10, retries))
generator.generate_by_part(split_size = 16,         
                           generate_size = 17,      # should be atleast same as split_size
                           max_retries = 20,
                           param_scheduler = param_scheduler,
                           info_depth = 15,         # lower makes it faster but may fail on difficult input
                           do_double_check = 10,    # give it a number to skip double check after X retries
                           loosen_restrict = 5,     # loosen restrictions
                           verbose_level = 1,       # how much log it displays
                           verbose_after_retries = 30)

In [ ]:
# map generated terrain back to sc values
result_shrinked = generator.get_result()
result_gen = shrinked_to_gen[result_shrinked]
tile_results = gen_to_sc[result_gen]

# randomize subtiles
if merge_subtiles:
    randomize_subtiles(tile_results, tileset, cv5_data = cv5_data)

In [ ]:
import time

# save generated chk
new_map_data = get_default_output_map_data()
chk_mtxm_data = tile_results.flatten().tolist()
dim, output_data = replace_tile_data(new_map_data, tileset, size, chk_mtxm_data)
with open("maps/generated_{}_{:04d}.chk".format(time.strftime("%Y%m%d_%H%M%S"), np.random.randint(1e4)), "wb") as fp:
    fp.write(output_data)

In [ ]:
from wfd.scmap import show_map, get_jpg
from IPython.display import display, Image as ImageIPD
show_map(tile_results, tileset, display_handler = lambda img: display(ImageIPD(data=get_jpg(img))))